In [ ]:
using Fermions
using Fermions.Propagators
using LinearAlgebra
using SparseIR
using Plots

In [ ]:
β = 3.0
U = 1.3
μ = U/2
ϵ = [#-9.955627166620991e-01,
     -3.631194257431713e-01,
     -9.049035188745458e-02,
     -3.291563979729537e-10,
      9.049035017337467e-02,
      3.631194218303095e-01,
      #9.955627128494740e-01
      ]
Vsq = [#0.2443372816032321,
      0.1785595949155028,
      0.0644602879943168,
      0.0240136764257886,
      0.0644602872543663,
      0.1785595941280526,
      #0.2443372834850736
      ]
V = sqrt.(Vsq)

In [ ]:
ftau(tau::Real) = dot(-Vsq, Propagators.tau_kernel.(tau, -ϵ, β))

In [ ]:
τ = 0:β/100:β
plot(τ, ftau.(τ))

In [ ]:
fs = FockSpace(Orbitals(length(V) + 1), FermionicSpin())
c = annihilators(fs)
n = occupations(fs)
↑ = 1 // 2
↓ = -1 // 2
imp = length(V) + 1

In [ ]:
H = U * n[imp, ↑] * n[imp, ↓] - μ * (n[imp, ↑] + n[imp, ↓])
for i in 1:length(V)
    for σ in (↑, ↓)
        H += ϵ[i] * n[i, σ]
        H += V[i] * c[imp, σ]' * c[i, σ]
        H += V[i] * c[i, σ]' * c[imp, σ]
    end
end

In [ ]:
H_eig = HamiltonianEigen(H, NSzSet(fs), β);

In [ ]:
exp_n = only(tau_propagator(c[imp,↑]', c[imp,↑], [0.0], H_eig, β))
exp_docc = only(tau_propagator(n[imp,↑], n[imp,↓], [0.0], H_eig, β))
exp_zocc = only(tau_propagator(n[imp,↑] - 0.5*I, n[imp,↓] - 0.5*I, [0.0], H_eig, β)) + 0.25
@show exp_n, exp_docc, exp_xocc;

In [ ]:
²